# 0. Imports

In [27]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import classifiers
import importlib
import pickle
import os

# Normal Modules

## Load datasets

In [19]:
reactome = np.array(pd.read_csv("../../data/processed/metrics/reactome_apid_huri.csv", header=0, index_col=0))
print(reactome.shape)
disgenet_sca = np.array(pd.read_csv("../../data/processed/metrics/disgenet_sca_apid_huri.csv", header=0, index_col=0))
print(disgenet_sca.shape)
disgenet_conservative = np.array(pd.read_csv("../../data/processed/metrics/disgenet_conservative_apid_huri.csv", header=0, index_col=0))
print(disgenet_conservative.shape)

(17089, 426)
(17089, 204)
(17089, 300)


In [20]:
reactome_labels = pd.read_csv('../../data/processed/reactome_labels_apid_huri.csv', header=None)
#reactome_labels.index = pd.read_csv("../../data/processed/metrics/reactome_metrics_apid_huri.csv", header=0, index_col=0).index
#reactome_labels.columns = pd.read_csv("../../data/processed/metrics/reactome_metrics_apid_huri.csv", header=0, index_col=0).columns
print(reactome_labels.shape)
disgenet_sca_labels = pd.read_csv("../../data/processed/disgenet_sca_labels_apid_huri.csv", header=None)
print(disgenet_sca_labels.shape)
disgenet_conservative_labels = pd.read_csv("../../data/processed/disgenet_conservative_labels_apid_huri.csv", header=None)
print(disgenet_conservative_labels.shape)

(17089, 426)
(17089, 204)
(17089, 300)


In [23]:
reactome_fs = pd.read_csv("../../data/processed/fs/reactome_apid_huri.csv")
print(reactome_fs.shape)
display(reactome_fs.head(2))
disgenet_sca_fs = pd.read_csv("../../data/processed/fs/disgenet_sca_apid_huri.csv")
print(disgenet_sca_fs.shape)
disgenet_conservative_fs = pd.read_csv("../../data/processed/fs/disgenet_conservative_apid_huri.csv")
print(disgenet_conservative_fs.shape)

(426, 299)


,result.1,result.2,result.3,result.4,result.5,result.6,result.7,result.8,result.9,result.10,...,result.414,result.415,result.417,result.419,result.421,result.422,result.423,result.424,result.425,result.426
0,18.424554,0.125218,0.114776,0.165712,0.028801,0.229715,0.163529,0.128769,0.130604,0.102407,...,0.075334,0.012056,0.013855,0.029297,0.985202,0.709918,1.059249,0.975577,0.177692,0.220787
1,0.133560,7.985435,8.777931,0.057642,0.054015,0.135188,0.062448,0.084157,0.044868,0.133419,...,0.001542,0.104665,0.822205,0.070612,0.544571,0.426107,0.471597,0.407100,0.020350,0.049993


(204, 204)
(300, 300)


## Reactome

In [59]:
importlib.reload(classifiers)
clf_results, clf_results_probability, cv_results, clf_models = classifiers.logistic_classifier(reactome, reactome_fs, reactome_labels)

  0%|          | 0/20 [00:00<?, ?it/s]

In [60]:
x = cv_results.copy()
x['l'] = cv_results.predictor_indices.apply(len)

In [62]:
clf_results

,f_measure,precision,recall,mcc,tp,fp,fn,tn
0,1.000000,1.000000,1.000000,1.000000,13,0,0,3405
1,1.000000,1.000000,1.000000,1.000000,11,0,0,3407
2,1.000000,1.000000,1.000000,1.000000,11,0,0,3407
3,1.000000,1.000000,1.000000,1.000000,12,0,0,3406
4,0.800000,0.714286,0.909091,0.805137,10,4,1,3403
5,1.000000,1.000000,1.000000,1.000000,15,0,0,3403
6,1.000000,1.000000,1.000000,1.000000,23,0,0,3395
7,1.000000,1.000000,1.000000,1.000000,23,0,0,3395
8,1.000000,1.000000,1.000000,1.000000,23,0,0,3395
9,1.000000,1.000000,1.000000,1.000000,23,0,0,3395


In [ ]:
clf_results.to_csv('../../models/GAP-MINE/binary/disease_closeness_lgr.csv', index=False)

## Disgenet SCA

In [ ]:
importlib.reload(classifiers)
clf_results, clf_results_probability, cv_results, n_predictors, clf_models = classifiers.logistic_classifier(
    disgenet_sca, disgenet_sca_fs, disgenet_sca_labels
)

## Disgenet Conservative

In [ ]:
importlib.reload(classifiers)
clf_results, clf_results_probability, cv_results, n_predictors, clf_models = classifiers.logistic_classifier(
    disgenet_conservative, disgenet_conservative_fs, disgenet_conservative_labels
)